In [1]:
import os
import time
import tensorflow as tf
import numpy as np

In [2]:
LABEL_DIMENSIONS = 10

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()
TRAINING_SIZE = len(train_images)
TEST_SIZE = len(test_images)

4423680/4422102 [==============================] - 70s 16us/step


In [17]:
train_images = np.asarray(train_images, dtype=np.float32) /255.0
test_images = np.asarray(test_images, dtype=np.float32) /255.0

In [18]:
# Convert the images and add channels.
train_images = train_images.reshape(-1, 28, 28, 1)
test_images = test_images.reshape(-1, 28,28,1)

In [19]:
train_labels = tf.keras.utils.to_categorical(train_labels, LABEL_DIMENSIONS)
test_labels = tf.keras.utils.to_categorical(test_labels, LABEL_DIMENSIONS)

In [23]:
# Building a convolutional model with tf.keras functional API.
inputs = tf.keras.Input(shape=(28,28,1))
x= tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu') (inputs)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2)(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3),
activation='relu')(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2)(x)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3),
activation='relu')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
predictions = tf.keras.layers.Dense(LABEL_DIMENSIONS,
activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=predictions)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)              

In [24]:
optimizer = tf.keras.optimizers.SGD()
model.compile(loss='categorical_crossentropy',
optimizer=optimizer,
metrics=['accuracy'])

In [25]:
strategy = None
#strategy = tf.distribute.MirroredStrategy()
config = tf.estimator.RunConfig(train_distribute=strategy)

In [26]:
estimator = tf.keras.estimator.model_to_estimator(model,
config=config)

INFO:tensorflow:Using the Keras model provided.
Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\archi\\AppData\\Local\\Temp\\tmptbudqe14', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_ch

In [27]:
def input_fn(images, labels, epochs, batch_size):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((images,labels))
    # Shuffle, repeat and batch of examples.
    SHUFFLE_SIZE = 5000
    dataset = dataset.shuffle(SHUFFLE_SIZE).repeat(epochs).batch(batch_size)
    dataset = dataset.prefetch(None)
    # return the dataset.
    return dataset

In [28]:
BATCH_SIZE = 512
EPOCHS = 50
estimator_train_result = estimator.train(input_fn=lambda:input_fn(train_images, train_labels, epochs = EPOCHS, batch_size =BATCH_SIZE))
print(estimator_train_result)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='C:\\Users\\archi\\AppData\\Local\\Temp\\tmptbudqe14\\keras\\keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: C:\Users\archi\AppData\Local\Temp\tmptbudqe14\keras\keras_model.ckpt
INFO:tensorflow:Warm-starting variables only in TRAINABLE_VARIABLES.
INFO:tensorflow:Warm-started 10 variables.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\archi\A

INFO:tensorflow:Loss for final step: 2.302049.


In [29]:
estimator.evaluate(lambda: input_fn(test_images, test_labels, epochs=1, batch_size=BATCH_SIZE))

INFO:tensorflow:Calling model_fn.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-01-24T15:04:36Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\archi\AppData\Local\Temp\tmptbudqe14\model.ckpt-5860
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 3.40468s
INFO:tensorflow:Finished evaluation at 2021-01-24-15:04:39
INFO:tensorflow:Saving dict for global step 5860: accuracy = 0.1956, global_step = 5860, loss = 2.3149865
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5860: C:\Users\archi\AppData\Local\Temp\tmptbudqe14\model.ckpt-5860


{'accuracy': 0.1956, 'loss': 2.3149865, 'global_step': 5860}